# import libraries 

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import math 

# import data 

In [2]:
# !wget ?-O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
# !unzip -o -j moviedataset.zip 

In [3]:
movies = pd.read_csv("movies.csv")
ratings = pd.read_csv("ratings.csv")

# data cleaning 

In [4]:
movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')
movies['title'] = movies['title'].apply(lambda x: x.strip())
movies.head()

/tmp/ipykernel_90568/1226832952.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['title'] = movies.title.str.replace('(\(\d\d\d\d\))', '')


,movieId,title,genres
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji,Adventure|Children|Fantasy
2,3,Grumpier Old Men,Comedy|Romance
3,4,Waiting to Exhale,Comedy|Drama|Romance
4,5,Father of the Bride Part II,Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [6]:
ratings = ratings.drop('timestamp', 1)
ratings.head()

/tmp/ipykernel_90568/1184197411.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  ratings = ratings.drop('timestamp', 1)


,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


# Collaborative Filtering

In [7]:
def find_input_id (movies,rating_movies):
    return  movies[movies['title'].isin(rating_movies['title'].tolist())]

In [8]:
def find_users_ratings (ratings , user_movies):
    return ratings[ratings['movieId'].isin(user_movies['movieId'].tolist())]

In [9]:
def find_best_users_match(ratings,inputMovies):
    
    users_rating = find_users_ratings(ratings,inputMovies)
    
    userGroups = users_rating.groupby(['userId'])
    
    userGroups = sorted(userGroups,  key=lambda x: len(x[1]), reverse=True)
    
    return userGroups[0:100]

In [35]:
def find_similarity_of_users(users,inputMovies):
    difference = dict()
    
    for user_id , rates in users:
        
        input_rates = inputMovies[inputMovies['movieId'].isin(rates['movieId'].tolist())]
        selected_rates = rates.sort_values(by='movieId')
        input_rates = input_rates.sort_values(by='movieId')
        inputRatingList = input_rates['rating'].tolist()
        selectedRatingList = selected_rates['rating'].tolist()
        
        difference[user_id] = sum([pow((inputRatingList[i]-selectedRatingList[i]),2) for i in range(len(inputRatingList))])
        
    pearsonDF = pd.DataFrame.from_dict(difference, orient='index')
    pearsonDF.columns = ['similarityIndex']
    pearsonDF['userId'] = pearsonDF.index
    
    return pearsonDF.sort_values(by='similarityIndex', ascending=False)[0:50]
        

In [36]:
def collaborative_filtering (user_rating , movies, ratings):
    
    inputId = find_input_id(movies,user_rating)
    inputMovies = pd.DataFrame(userInput)
    inputMovies = pd.merge(inputId, inputMovies)
    
    
    users = find_best_users_match(ratings,inputMovies)
    similar_users = find_similarity_of_users(users,user_rating)
    
    topUsersRating= similar_users.merge(ratings, left_on='userId', right_on='userId', how='inner')
    topUsersRating['weightedRating'] = topUsersRating['similarityIndex']*topUsersRating['rating']
    tempTopUsersRating = topUsersRating.groupby('movieId').sum()[['similarityIndex','weightedRating']]
    tempTopUsersRating.columns = ['sum_similarityIndex','sum_weightedRating']
    
    recommendation_df = pd.DataFrame()
    recommendation_df['weighted average recommendation score'] = tempTopUsersRating['sum_weightedRating']/tempTopUsersRating['sum_similarityIndex']
    recommendation_df['movieId'] = tempTopUsersRating.index
    recommendation_df = recommendation_df.sort_values(by='weighted average recommendation score', ascending=False)
    return movies.loc[movies['movieId'].isin(recommendation_df.head(10)['movieId'].tolist())]
    
    

In [37]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
inputId = movies[movies['title'].isin(inputMovies['title'].tolist())]
inputMovies = pd.merge(inputId, inputMovies)



In [38]:
collaborative_filtering (inputMovies , movies, ratings)

,movieId,title,genres
799,1046,Beautiful Thing,Drama|Romance
844,1111,Microcosmos (Microcosmos: Le peuple de l'herbe),Documentary
5236,8580,Into the Woods,Adventure|Comedy|Fantasy|Musical
7364,78836,Enter the Void,Drama
7380,79274,Batman: Under the Red Hood,Action|Animation
7480,82242,Rare Exports: A Christmas Tale (Rare Exports),Action|Comedy
7491,82744,Faster,Action|Crime|Drama
7521,84273,Zeitgeist: Moving Forward,Documentary
7524,84414,All-Star Superman,Action|Adventure|Animation|Sci-Fi
9147,147376,Doctor Who: A Christmas Carol,Sci-Fi
